In [108]:
# 1b, get the unique id of the bus stop first
import json
import requests
northIdFounded = False
southIdFounded = False

for rc in ['11S','11M']:
    api = "https://data.etagmb.gov.hk/route/NT/" + rc
    res = requests.get(api)
    info = res.json()
    info = info['data']
    routeID = info[0]["route_id"]
    dir = info[0]['directions']

    for i in range(len(dir)):
        routeSeq = dir[i]['route_seq']
        apiTwo = 'https://data.etagmb.gov.hk/route-stop/' + str(routeID) + '/' + str(routeSeq)
        resTwo = requests.get(apiTwo)
        resTwo = resTwo.json()
        routeStops = resTwo['data']['route_stops']
        for j in range(len(routeStops)):
            if not southIdFounded:
                if any('香港科技大学(南站)' in str(value) for value in routeStops[j].values()):
                    southID = routeStops[j]["stop_id"]
                    southIDFounded = True
            if not northIdFounded:
                if any('香港科技大学(北站)' in str(value) for value in routeStops[j].values()):
                    northID = routeStops[j]["stop_id"]
                    northIdFounded = True
            if southIdFounded and northIdFounded:
                break
        if southIdFounded and northIdFounded:
                break
    if southIdFounded and northIdFounded:
                break

print(f'The Stop id for HKUST(N) is {northID}')
print(f'The Stop id for HKUST(S) is {southID}')
            
            
    # print(json.dumps(info, sort_keys=True, indent=4))
    # print(routeID)
    # break


The Stop id for HKUST(N) is 20012474
The Stop id for HKUST(S) is 20013011


In [110]:
# 1a. Get route id
routeIDList = []
for stopID in [northID, southID]:
    apiToVisit = 'https://data.etagmb.gov.hk/stop-route/' + str(stopID)
    stopInfo = requests.get(apiToVisit)
    stopInfo = stopInfo.json()
    data = stopInfo['data']
    for i in range(len(data)):
        id = data[i]['route_id']
        if not id in routeIDList:
            routeIDList.append(id)
print(routeIDList)

[2004764, 2004825, 2004826, 2004827, 2004828]


In [112]:
# 1a. Get route code
routeCodeList = []
descriptionList = []
for ri in routeIDList:
    api = 'https://data.etagmb.gov.hk/route/' + str(ri)
    rc = requests.get(api)
    rc = rc.json()
    routeCode = rc['data']
    routeCode = routeCode[0]
    description = routeCode['description_en']
    routeCode = routeCode['route_code']
    routeCodeList.append(routeCode)
    descriptionList.append(description)
print('Minibus that has a stop at HKUST:')
print(routeCodeList)
print(descriptionList)

Minibus that has a stop at HKUST:
['12', '11M', '11S', '11B', '11B']
['Normal Route', 'Normal Route', 'Overnight Service Route', 'Morning special trips Route ', 'Evening special trips Route ']


In [120]:
import requests
import pandas as pd
from datetime import datetime


excludeID = []
includeID = []
excludeSeq = []
dest = []
print(routeIDList)
my_dict = []
myDictDest = []
for ri in routeIDList:
    api = 'https://data.etagmb.gov.hk/route/' + str(ri)
    rc = requests.get(api)
    rc = rc.json()
    data = rc['data']
    data = data[0]
    direction = data['directions']
    for j in range(len(direction)):
        if any('香港科技大学' in str(value) for value in direction[j].values()):
            my_dict.append({
                            'excludeID': ri,
                            'excludeSeq': direction[j]['route_seq']
                        })
            # excludeID.append(ri)
            # excludeSeq.append(direction[j]['route_seq'])
        else:
            myDictDest.append({
                            'includeID': ri,
                            'dest': direction[j]['dest_en']
                        })
            # includeID.append(ri)
            # dest.append(direction[j]['dest_en'])
    
myDict = dict(zip(routeIDList, routeCodeList))


print(myDictDest)
def get_timetable(direction):
    if direction not in ["N", "S"]:
        print("No such direction")
        return
    
    stop_id = northID if direction == "N" else southID
    timetable = []

    api_url = f'https://data.etagmb.gov.hk//eta/stop/{stop_id}'
    response = requests.get(api_url)
        
    if response.status_code == 200:
        data = response.json()['data']
        for bus in data:
            # Avoid buses that terminate at HKUST
            routeID = bus['route_id']
            seq = bus['route_seq']

            flag = False
            for item in my_dict:
                if item['excludeID'] == routeID and item['excludeSeq'] == seq:
                    flag = True
                        
            if not flag:
                eta = bus['eta']
                for i in range(len(eta)):

                    time = datetime.fromisoformat(eta[i]['timestamp'][:-6])  # 去掉最后的时区部分
                    formatted_time = time.strftime('%H:%M')  # 格式化为 HH:MM

                    for item in myDictDest:
                        if item['includeID'] == routeID:
                            dest_en = item['dest']
                    timetable.append({
                        'route_code': myDict[routeID],
                        'dest_en': dest_en,
                        'ETA': formatted_time
                    })

    # Create a DataFrame and sort by ETA
    df = pd.DataFrame(timetable)
    df = df.sort_values(by='ETA')
    
    # Display the timetable
    print(df.reset_index(drop=True))

# Example usage
get_timetable("N")  # For North gate
get_timetable("S")  # For South gate

[2004764, 2004825, 2004826, 2004827, 2004828]
[{'includeID': 2004764, 'dest': 'Po Lam'}, {'includeID': 2004764, 'dest': 'Sai Kung'}, {'includeID': 2004826, 'dest': 'Choi Hung Station (Lung Cheung Road)'}, {'includeID': 2004826, 'dest': 'Po Lam'}]
  route_code dest_en    ETA
0        11S  Po Lam  01:39
1        11S  Po Lam  02:09
  route_code dest_en    ETA
0        11S  Po Lam  01:41
1        11S  Po Lam  02:11


In [ ]:
# 2a
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import csv
from datetime import datetime, timedelta

# 设置 ChromeDriver 路径
service = Service('C:/Users/user/Desktop/chromedriver-win32/chromedriver.exe')

# 设置 Chrome 无头模式
chrome_options = Options()
chrome_options.add_argument("--headless")  # 启用无头模式
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 启动浏览器
driver = webdriver.Chrome(service=service, options=chrome_options)

# # 启动浏览器
# driver = webdriver.Chrome(service=service)

# # 打开目标网页
# url = "https://www.hko.gov.hk/en/cis/dailyExtract.htm?y=2024&m=11"
# driver.get(url)

# 初始化日期范围
start_date = datetime(2004, 11, 1)
end_date = datetime(2024, 10, 31)
current_date = start_date

# 用于存储所有数据的列表
all_data = []

try:
    while current_date <= end_date:
        # 构建 URL
        url = f"https://www.hko.gov.hk/en/cis/dailyExtract.htm?y={current_date.year}&m={current_date.month}"
        driver.get(url)

        # 等待表格加载
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'dataTable'))  # 确保表格元素已加载
        )
        
        # 查找表格
        table = driver.find_element(By.ID, 'dataTable')  # 替换为实际表格 ID
        
        # 提取数据
        rows = table.find_elements(By.TAG_NAME, 'tr')
        # rows = rows[:-2]

        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if cells:
                # 提取第一列（日期）并转换格式
                day = cells[0].text.strip()
                rainfall = cells[8].text.strip()
                if rainfall == "Trace":
                    rainfall = 0.02
                if day:  # 确保有数据
                    try:
                        # 尝试将日期转换为字符串
                        date_str = f"{current_date.year}-{current_date.month:02d}-{int(day):02d}"
                        all_data.append([date_str] + [cell.text.strip() for cell in cells[1:8]] + [rainfall])  # 添加前9列数据
                    except ValueError:
                        # 如果转换失败，跳过该行
                        continue
                    # 构建日期字符串
                    # date_str = f"{current_date.year}-{current_date.month:02d}-{int(day):02d}"

                    # all_data.append([date_str] + [cell.text.strip() for cell in cells[1:8]] + [rainfall])  # 添加前9列数据

        # 移动到下一个月
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    # 导出为 CSV 文件
    with open('weather.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(all_data)

    print("数据已成功导出为 weather.csv")
except Exception as e:
    print(f"发生错误: {e}")

finally:
    # 关闭浏览器
    driver.quit()

In [ ]:
# 2b, hot weather
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# 设置 ChromeDriver 路径
service = Service('C:/Users/user/Desktop/chromedriver-win32/chromedriver.exe')

# 启动浏览器
driver = webdriver.Chrome(service=service)

try:
    # 打开目标网页
    url = "https://www.hko.gov.hk/en/wxinfo/climat/warndb/warndb13.shtml"
    driver.get(url)

    # 等待页面加载
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'startdate'))  # 修改为正确的ID
    )

    # 输入年份和月份
    start_date_input = driver.find_element(By.ID, 'startdate')
    end_date_input = driver.find_element(By.ID, 'enddate')

    # 清除输入框中的内容
    start_date_input.clear()
    end_date_input.clear()

    # 设置要查询的起始和结束时间
    start_date = "200411"  # 修改为所需的开始日期
    end_date = "202411"    # 修改为所需的结束日期

    start_date_input.send_keys(start_date)
    end_date_input.send_keys(end_date)

    # 提交查询
    submit_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@value='Submit Query']"))
    )
    submit_button.click()

    # 等待表格加载
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'result'))  # 确保表格元素已加载
    )

    # 查找表格
    table = driver.find_element(By.ID, 'result')

    # 提取数据
    rows = table.find_elements(By.TAG_NAME, 'tr')

    # 用于存储所有数据的列表
    all_data = []

    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        if cells:
            # 提取时间和日期，去掉第五列
            time_str = cells[0].text.strip()  # HH MM
            date_str = cells[1].text.strip()  # DD/MON/YYYY
            time_strEnd = cells[2].text.strip()  # HH MM
            date_strEnd = cells[3].text.strip()  # DD/MON/YYYY


            # 转换为 'yyyy-mm-dd hh:mm:ss' 格式
            day, month, year = date_str.split('/')
            dayEnd, monthEnd, yearEnd = date_strEnd.split('/')
            month_num = {
                "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
                "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
                "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
            }[month]
            month_numEnd = {
                "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
                "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
                "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
            }[monthEnd]

            hour, minute = time_str.split(':')
            hourEnd, minuteEnd = time_strEnd.split(':')
            datetime_str = f"{year}-{month_num}-{day} {hour}:{minute}:00"
            datetime_strEnd = f"{yearEnd}-{month_numEnd}-{dayEnd} {hourEnd}:{minuteEnd}:00"
            all_data.append([datetime_str] + [datetime_strEnd])

    # 导出为 CSV 文件
    with open('hot.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(all_data)

    print("数据已成功导出为 hot.csv")

except Exception as e:
    print(f"发生错误: {e}")
    print(driver.page_source)  # 打印当前页面的HTML内容以调试

finally:
    # 关闭浏览器
    driver.quit()

In [ ]:
# 2b, cold weather
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# 设置 ChromeDriver 路径
service = Service('C:/Users/user/Desktop/chromedriver-win32/chromedriver.exe')

# 启动浏览器
driver = webdriver.Chrome(service=service)

try:
    # 打开目标网页
    url = "https://www.hko.gov.hk/en/wxinfo/climat/warndb/warndb12.shtml"
    driver.get(url)

    # 等待页面加载
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'startdate'))  # 修改为正确的ID
    )

    # 输入年份和月份
    start_date_input = driver.find_element(By.ID, 'startdate')
    end_date_input = driver.find_element(By.ID, 'enddate')

    # 清除输入框中的内容
    start_date_input.clear()
    end_date_input.clear()

    # 设置要查询的起始和结束时间
    start_date = "200411"  # 修改为所需的开始日期
    end_date = "202411"    # 修改为所需的结束日期

    start_date_input.send_keys(start_date)
    end_date_input.send_keys(end_date)

    # 提交查询
    submit_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@value='Submit Query']"))
    )
    submit_button.click()

    # 等待表格加载
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'result'))  # 确保表格元素已加载
    )

    # 查找表格
    table = driver.find_element(By.ID, 'result')

    # 提取数据
    rows = table.find_elements(By.TAG_NAME, 'tr')

    # 用于存储所有数据的列表
    all_data = []

    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        if cells:
            # 提取时间和日期，去掉第五列
            time_str = cells[0].text.strip()  # HH MM
            date_str = cells[1].text.strip()  # DD/MON/YYYY
            time_strEnd = cells[2].text.strip()  # HH MM
            date_strEnd = cells[3].text.strip()  # DD/MON/YYYY


            # 转换为 'yyyy-mm-dd hh:mm:ss' 格式
            day, month, year = date_str.split('/')
            dayEnd, monthEnd, yearEnd = date_strEnd.split('/')
            month_num = {
                "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
                "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
                "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
            }[month]
            month_numEnd = {
                "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
                "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
                "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
            }[monthEnd]

            hour, minute = time_str.split(':')
            hourEnd, minuteEnd = time_strEnd.split(':')
            datetime_str = f"{year}-{month_num}-{day} {hour}:{minute}:00"
            datetime_strEnd = f"{yearEnd}-{month_numEnd}-{dayEnd} {hourEnd}:{minuteEnd}:00"
            all_data.append([datetime_str] + [datetime_strEnd])

    # 导出为 CSV 文件
    with open('cold.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(all_data)

    print("数据已成功导出为 cold.csv")

except Exception as e:
    print(f"发生错误: {e}")
    print(driver.page_source)  # 打印当前页面的HTML内容以调试

finally:
    # 关闭浏览器
    driver.quit()

In [ ]:
# 2c
import pandas as pd

# 读取CSV文件
weather_df = pd.read_csv('weather.csv', header=None)
hot_df = pd.read_csv('hot.csv', header=None)
cold_df = pd.read_csv('cold.csv', header=None)

# 重命名列
weather_df.columns = ['Date', 'Mean Pressure', 'Absolute Daily Max Air Temp.', 'Mean Air Temp.', 'Absolute Daily Min Air Temp.', 'Mean Dew Point', 'Mean Relative Humidity', 'Mean Amount of Cloud', 'Total Rainfall']
hot_df.columns = ['Hot_Start', 'Hot_End']
cold_df.columns = ['Cold_Start', 'Cold_End']

# 转换日期格式
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
hot_df['Hot_Start'] = pd.to_datetime(hot_df['Hot_Start'])
hot_df['Hot_End'] = pd.to_datetime(hot_df['Hot_End'])
cold_df['Cold_Start'] = pd.to_datetime(cold_df['Cold_Start'])
cold_df['Cold_End'] = pd.to_datetime(cold_df['Cold_End'])

# 初始化新列
weather_df['Very Hot Weather Warning'] = False
weather_df['Cold Weather Warning'] = False

# 标记Very Hot Weather Warning
for _, row in hot_df.iterrows():
    mask = (weather_df['Date'] >= row['Hot_Start']) & (weather_df['Date'] <= row['Hot_End'])
    weather_df.loc[mask, 'Very Hot Weather Warning'] = True

# 标记Cold Weather Warning
for _, row in cold_df.iterrows():
    mask = (weather_df['Date'] >= row['Cold_Start']) & (weather_df['Date'] <= row['Cold_End'])
    weather_df.loc[mask, 'Cold Weather Warning'] = True

# 保存结果
weather_df.to_csv('weather_with_warnings.csv', index=False)

In [ ]:
# 2d, hot
import pandas as pd
import matplotlib.pyplot as plt

# 读取数据
data = pd.read_csv('weather_with_warnings.csv')

# 将数据分为有高温天气警告和没有高温天气警告的两组
hot_warning = data[data['Very Hot Weather Warning'] == True]
no_hot_warning = data[data['Very Hot Weather Warning'] == False]

# 设置参数列表
parameters = [
    'Mean Pressure',
    'Absolute Daily Max Air Temp.',
    'Mean Air Temp.',
    'Absolute Daily Min Air Temp.',
    'Mean Dew Point',
    'Mean Relative Humidity',
    'Mean Amount of Cloud',
    'Total Rainfall'
]

# 创建2x4的子图
fig, axes = plt.subplots(2, 4, figsize=(15, 10))
axes = axes.flatten()  # 将2D数组展平，方便迭代

# 为每个参数绘制直方图
for i, param in enumerate(parameters):
    axes[i].hist(no_hot_warning[param], bins=20, alpha=0.5, label='No Hot Warning', color='red')
    axes[i].hist(hot_warning[param], bins=20, alpha=0.5, label='Hot Warning', color='blue')

    axes[i].set_title(param)
    axes[i].set_xlabel(param)
    axes[i].set_ylabel('Number of Days')
    axes[i].legend()

# 调整布局
plt.tight_layout()
plt.show()

In [ ]:
# 2d, cold
import pandas as pd
import matplotlib.pyplot as plt

# 读取数据
data = pd.read_csv('weather_with_warnings.csv')

# 将数据分为有高温天气警告和没有高温天气警告的两组
cold_warning = data[data['Cold Weather Warning'] == True]
no_cold_warning = data[data['Cold Weather Warning'] == False]

# 设置参数列表
parameters = [
    'Mean Pressure',
    'Absolute Daily Max Air Temp.',
    'Mean Air Temp.',
    'Absolute Daily Min Air Temp.',
    'Mean Dew Point',
    'Mean Relative Humidity',
    'Mean Amount of Cloud',
    'Total Rainfall'
]

# 创建2x4的子图
fig, axes = plt.subplots(2, 4, figsize=(15, 10))
axes = axes.flatten()  # 将2D数组展平，方便迭代

# 为每个参数绘制直方图
for i, param in enumerate(parameters):
    axes[i].hist(no_cold_warning[param], bins=20, alpha=0.5, label='No Cold Warning', color='red')
    axes[i].hist(cold_warning[param], bins=20, alpha=0.5, label='Cold Warning', color='blue')

    axes[i].set_title(param)
    axes[i].set_xlabel(param)
    axes[i].set_ylabel('Number of Days')
    axes[i].legend()

# 调整布局
plt.tight_layout()
plt.show()